## Liver-related laboratory data after receiving the second dose of medication.

In [ ]:
import root_config as rc

rc.configure()

from detectdd.auth_bigquery import BigQueryClient
from detectdd.serializer import Serializer

try:
    cohort = Serializer().read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")


big_query = BigQueryClient.auth()

cohort.describe()

In [ ]:
icu = 'physionet-data.mimiciv_icu'

In [ ]:
# keywords = ["creatinine", "bun", "uric acid"]


sql = f"""
SELECT itemid, label 
FROM `{icu}.d_items` 
WHERE label LIKE '%ALT%'
"""
query_job = big_query.query(sql)
results = query_job.result()

for row in results:
    print(row.itemid, row.label)


In [ ]:
sql = f"""
SELECT itemid, label 
FROM `{icu}.d_items` 
WHERE label LIKE '%AST%'
"""
query_job = big_query.query(sql)
results = query_job.result()

for row in results:
    print(row.itemid, row.label)

In [ ]:
sql = f"""
SELECT itemid, label 
FROM `{icu}.d_items` 
WHERE label LIKE '%Alkaline%'
"""
query_job = big_query.query(sql)
results = query_job.result()

for row in results:
    print(row.itemid, row.label)

In [ ]:
from detectdd.query_multiplexer import QueryMultiplexer, WhereClauseGenerator
from detectdd import config

icu = 'physionet-data.mimiciv_icu'

selected_itemids = {
    "ALT": 220644,
    "AST": 220587,
    "Alkaline Phosphate": 225612
}

query_multiplexer = QueryMultiplexer(BigQueryClient().auth())

def fetch_data():
    sql_template = f"""SELECT ce.subject_id AS subject_id,
            ce.stay_id AS stay_id,
            ce.hadm_id AS hadm_id,
            ce.itemid AS item_id,
            ce.charttime AS chart_time,
            ce.valuenum AS value,
            CASE
                WHEN ce.itemid = 220644 THEN 'ALT'
                WHEN ce.itemid = 220587 THEN 'AST'
                WHEN ce.itemid = 225612 THEN 'Alkaline Phosphate'
            END AS parameter_name
        FROM `{icu}.chartevents` AS ce
        WHERE ce.itemid IN {tuple(selected_itemids.values())}
        AND ($where)"""
    
    where_fragment = "(ce.stay_id= $stay_id AND DATETIME_DIFF(ce.charttime, '$dose_b_time', HOUR) BETWEEN 0 AND 12)"

    multimap_data = {k: v.tolist() for k, v in cohort.groupby('stay_id')['dose_b_time']}
    results = query_multiplexer.multiplex_query(sql_template, multi_map_data=multimap_data, where_clause=WhereClauseGenerator(where_fragment, "stay_id", "dose_b_time"))
    
    return results

results = fetch_data()
print(results)


In [ ]:
results